In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import r2_score

In [ ]:
data_Train=pd.read_csv('/content/drive/MyDrive/dataset/polynomial_train.csv')
data_Train = data_Train.drop(columns='Unnamed: 0') 
data_Train

,A,B,C,label
0,-1113.924581,952.105404,-276.045881,-1.294289e+03
1,-2229.480228,1111.488568,-93.454107,-2.714687e+04
2,2717.257713,53.431237,-179.228591,6.474300e+04
3,-1638.783965,-632.919447,-106.170237,-7.349871e+03
4,5173.934338,-721.895577,-75.074840,1.651328e+06
...,...,...,...,...
49995,-401.109554,205.180269,-123.837422,-6.275985e+00
49996,832.536891,-723.312711,-128.736597,2.390843e+02
49997,-533.446993,1171.457335,-130.840925,-2.857080e+02
49998,1810.499103,-424.111696,-128.334986,8.161595e+03


In [ ]:
data_Test=pd.read_csv('/content/drive/MyDrive/dataset/polynomial_test_data.csv')
data_Test = data_Test.drop(columns='ids') 
data_Test

,A,B,C
0,-961.622191,789.966580,-43.660479
1,3460.005131,786.092527,-144.096657
2,1648.281427,241.985600,-267.040826
3,-5361.100419,1062.669779,-199.007043
4,-2303.208688,-248.185768,-317.468950
...,...,...,...
19995,-1941.831479,284.320247,-88.712478
19996,-993.524028,535.348652,-97.151458
19997,665.127361,683.355125,-184.768610
19998,-80.508468,-24.409192,-153.713418


In [ ]:
def normalize_data(x):
 mu=np.mean(x,axis=0)
 sigma=np.std(x,axis=0)
 x=(x-mu)/sigma
 return x

In [ ]:
X=data_Train.to_numpy()
m=X.shape[0]
n=(X.shape[1]-1)
X_Train=X[:,:n]
print(X_Train.shape)

X_Train=normalize_data(X_Train)
print(X_Train)
Y_Train=X[:,n]
Y_Train=Y_Train.reshape(m,1)
print(Y_Train.shape)
print(Y_Train)



(50000, 3)
[[-0.41384769  0.98132788 -1.77549966]
 [-0.80507801  1.20523369  0.78992095]
 [ 0.92976497 -0.28115405 -0.41521346]
 ...
 [-0.21027162  1.28947944  0.2646349 ]
 [ 0.6117607  -0.95201936  0.29984342]
 [-0.32546513  0.00524071 -0.37157799]]
(50000, 1)
[[ -1294.28891782]
 [-27146.87008356]
 [ 64743.00171817]
 ...
 [  -285.70796526]
 [  8161.59530383]
 [  -255.16713134]]


In [ ]:
X_Test=data_Test.to_numpy()
m=X_Test.shape[0]
n=(X_Test.shape[1])
X_Test=X_Test[:,:n]
print(X_Test.shape)
print(X_Test)


(20000, 3)
[[ -961.62219071   789.96658017   -43.66047882]
 [ 3460.00513094   786.0925266   -144.09665669]
 [ 1648.28142716   241.98559957  -267.04082638]
 ...
 [  665.12736087   683.35512523  -184.76861036]
 [  -80.50846817   -24.40919225  -153.71341796]
 [-2997.29890736   778.62209693   -59.34016472]]


In [ ]:
class NeuralNetwork:
    def init_params(architecture):
     w=[]
     b=[]
     for i in range(architecture.shape[1]-1):
         a=np.random.rand(architecture[0][i],architecture[0][i+1])
        # *math.sqrt(2/(architecture[0][i+1]))     
         w.append(a)
     for i in range(architecture.shape[1]-1):
         c=np.random.rand(1,architecture[0][i+1])
        # *math.sqrt(2/(architecture[0][i+1]))
         b.append(c)   

     return w,b



    def sigmoid(input):    
     output = 1 / (1 + np.exp(-input))
     return output



    def sigmoid_derivative(input):
     return NeuralNetwork.sigmoid(input) * (1.0 - NeuralNetwork.sigmoid(input))

      
      
      
    def forward_prop(w,b,X,architecture):
     all_activations=[]
     z_all=[]
     x=X
     for i in range(architecture.shape[1]-2):
       z=np.dot(x,w[i])+b[i]
    # print(z.shape)
       z_all.append(z)
       a=NeuralNetwork.sigmoid(z)
    #  print(a.shape)
       x=a
    #  print(x.shape)
       all_activations.append(a)

     y_predicted=np.dot(x,w[architecture.shape[1]-2])+b[architecture.shape[1]-2]
     z_all.append(y_predicted)
    # print(ypred.shape) 
    # activation = np.exp(ypred) / np.sum(np.exp(ypred),axis=0)
    # print(activation.shape)
     all_activations.append(y_predicted)
    # y_predicted=np.argmax(activation,axis=1,keepdims = True)
    # print(y_predicted.shape)
     return y_predicted,all_activations,z_all





    def backward_prop(X,Y,architecture,w,b,all_activations,z):
      m=X.shape[0]
      derivative_w=[]
      derivative_b=[]
      a=all_activations[-1]-Y
      loss=(1/2*m)*np.sum(np.square(a))

      # lossarray=np.multiply(all_activations[-1],Y)
      # lossarray[np.where(lossarray==0)]=1
      # log=np.log(lossarray)
      # loss=np.sum(log)*(-1/m)

      dz=a
      dW = 1 / m * np.dot(all_activations[-2].T,dz)
      db = 1 / m * np.sum(dz,axis=0,keepdims = True)
      # print(dz.shape)
      # print(dW.shape)
      # print(db.shape)
      derivative_w.append(dW)
      derivative_b.append(db)
      
      for i in range(architecture.shape[1]-3):
        dZi = np.dot(dz,w[architecture.shape[1]-2-i].T) * NeuralNetwork.sigmoid_derivative(z[architecture.shape[1]-3-i])
        # print(dZi.shape)
        dwi= 1 / m * np.dot(all_activations[architecture.shape[1]-4-i].T,dZi)
        # print(dwi.shape)
        dbi= 1 / m * np.sum(dZi,axis=0,keepdims = True)
        # print(dbi.shape)
        derivative_w.append(dwi)
        derivative_b.append(dbi)
        dz=dZi
    #   print(dZi.shape)
    #   print(dwi.shape)
    #   print(dbi.shape)
       
      dz1=np.dot(dz,w[1].T) * NeuralNetwork.sigmoid_derivative(z[0])
      dW1 = 1 / m * np.dot(X.T,dz1)
      db1= 1 / m * np.sum(dz1,axis=0,keepdims = True)
      # print(dz1.shape)
      # print(dW1.shape)       
      # print(db1.shape)
      derivative_w.append(dW1)
      derivative_b.append(db1)
      derivative_w.reverse()
      derivative_b.reverse()
      return derivative_w,derivative_b,loss
 

    def update_params(w,b,dw,db,alpha,a):
      for i in range(a.shape[1]-1):
        w[i] = w[i] - (alpha*dw[i])
        b[i] = b[i] - (alpha*db[i])

      return w, b


    def gradient_descent(X, Y, alpha, iterations,architecture):
      losses=[]
      w,b=NeuralNetwork.init_params(architecture)
      for i in range(iterations):
        y_hat,act,zall=NeuralNetwork.forward_prop(w,b,X,architecture)
        DW,DB,loss=NeuralNetwork.backward_prop(X,Y,architecture,w,b,act,zall)
        w_updated,b_updated=NeuralNetwork.update_params(w,b,DW,DB,alpha,architecture)
        w=w_updated
        b=b_updated
        print("after",i,"iterations")
        print(loss)
        # NeuralNetwork.get_accuracy(y_hat,Y)
        losses.append(loss)

         
      return w,b,losses


    def get_accuracy(y__predicted,y):
        print(r2_score(y,y__predicted))


         
    def accPlot(loss, iters):
      plt.plot(np.arange(iters), loss, '-b') 
      plt.xlabel('Number of iterations')
      plt.ylabel('loss')
      plt.show()

In [ ]:
a=np.array([[3,20,5,1]])
alpha=0.01
iterations=100
w_final,b_final,loss_arr=NeuralNetwork.gradient_descent(X_Train,Y_Train,alpha,iterations,a)

after 0 iterations
8.344859432980821e+21
after 1 iterations
8.34478087012149e+21


<ipython-input-19-66f6b8ab691f>:19: RuntimeWarning: overflow encountered in exp
  output = 1 / (1 + np.exp(-input))


after 2 iterations
8.34485844158227e+21
after 3 iterations
8.344857899835634e+21
after 4 iterations
8.344857368869755e+21
after 5 iterations
8.344856848470098e+21
after 6 iterations
8.344856338426393e+21
after 7 iterations
8.34485583853256e+21
after 8 iterations
8.344855348586611e+21
after 9 iterations
8.344854868390589e+21
after 10 iterations
8.344854397750467e+21
after 11 iterations
8.344853936476081e+21
after 12 iterations
8.344853484381059e+21
after 13 iterations
8.344853041282726e+21
after 14 iterations
8.344852607002051e+21
after 15 iterations
8.344852181363562e+21
after 16 iterations
8.344851764195278e+21
after 17 iterations
8.344851355328644e+21
after 18 iterations
8.344850954598453e+21
after 19 iterations
8.344850561842795e+21
after 20 iterations
8.344850176902973e+21
after 21 iterations
8.344849799623456e+21
after 22 iterations
8.344849429851799e+21
after 23 iterations
8.344849067438598e+21
after 24 iterations
8.344848712237422e+21
after 25 iterations
8.344848364104749e+21
af